In [49]:
from elasticsearch_dsl import connections
from elasticsearch import Elasticsearch

# ??connections.create_connection
# ??Elasticsearch

connections.create_connection(hosts=['localhost:9200'], timeout=20)

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [50]:
from elasticsearch_dsl import Document, Date, Integer, Keyword, Text

class Person(Document):
    first_name = Text(analyzer='snowball')
    last_name = Text(analyzer='snowball')
    gender = Text(analyzer='snowball')
    age = Integer()

    class Index:
        name = 'person'
        settings = {
          'number_of_shards': 2,
        }

    def save(self, ** kwargs):
        return super(Person, self).save(**kwargs)

In [51]:
Person.init()

In [52]:
p = Person(
    meta={'id': 1},
    first_name='John',
    last_name='Smith',
    gender='M',
    age=22
)
p.save()

'updated'

In [53]:
p = Person(
    meta={'id': 2},
    first_name='James',
    last_name='Smith',
    gender='M',
    age=23
)
p.save()

'updated'

In [54]:
p.to_dict()

{'first_name': 'James', 'last_name': 'Smith', 'gender': 'M', 'age': 23}

In [55]:
p = Person.get(id=1)
p.meta, p.first_name, p.last_name, p.gender, p.age, p.meta.id

({'index': 'person', 'id': '1', 'version': 3, 'seq_no': 5, 'p...},
 'John',
 'Smith',
 'M',
 22,
 '1')

In [56]:
Person.mget([1, 2, 3], missing='skip')

[Person(index='person', id='1'), Person(index='person', id='2')]

In [30]:
p = Person.get(id=2)
p.delete()

In [65]:
from elasticsearch_dsl import Search

s = Search(index='person')
s.update_from_dict({'from': 0, 'size': 1, 'query': {'match_all': {}}})

hits = s.execute()
print(len(hits))

for hit in hits:
    print(type(hit), hit.meta.id, Person(meta={'id': hit.meta.id}, **hit.to_dict()))
    

1
<class 'elasticsearch_dsl.response.hit.Hit'> 1 Person(id='1')


In [62]:
hits[0]

<Hit(person/1): {'first_name': 'John', 'last_name': 'Smith', 'gender': 'M', ...}>

In [41]:
hits[0:2]

[<Hit(person/1): {'first_name': 'John', 'last_name': 'Smith', 'gender': 'M', ...}>, <Hit(person/2): {'first_name': 'James', 'last_name': 'Smith', 'gender': 'M',...}>]

In [22]:
connections.get_connection().cluster.health()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 5,
 'active_shards': 5,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 71.42857142857143}